# Batch processing of single cell images for full image analysis pipeline

In [1]:
from aicsimageio import AICSImage
from skimage.feature import peak_local_max
import pyclesperanto_prototype as cle
import napari
import os
import numpy as np
import pandas as pd

## Functions

In [2]:
def segmentation(image, threshold, zoom_factor):
    
    #Noise removal
    g_blur = cle.gaussian_blur(image, sigma_x=1, sigma_y=1, sigma_z=1)
    
    #background subtracton
    background_subtracted = np.asarray(cle.top_hat_box(g_blur, radius_x=2*zoom_factor, radius_y=2*zoom_factor, radius_z=2*zoom_factor))
    del g_blur
    
    #threshold = threshold_otsu(background_subtracted)
    peaks = peak_local_max(background_subtracted, threshold_abs=threshold, min_distance=2)
    
    # Find local peaks above the specified threshold
    spots = cle.create(image.shape, dtype=np.uint32)
    peaks = peaks[:,::-1]
        
    # Convert the list of peaks to a labeled image
    labeled_spots = cle.pointlist_to_labelled_spots(peaks.T, spots)
    
    # Expand the labeled spots by a certain radius
    expanded_labels = cle.dilate_labels(labeled_spots, radius=5)
    
    #Create a binary mask where all pixels above half of the maximum intensity in each labels are 1
    #This is done to perform full-width half-maxima for estimating foci boundary
    threshold_map = cle.maximum_intensity_map(background_subtracted, expanded_labels) / 2
    thresholded_image = np.asarray(background_subtracted) > np.asarray(threshold_map)
    
    labeled_foci = (expanded_labels * thresholded_image).astype(np.uint32)
    
    #Exclude small labels
    output_labels = cle.exclude_small_labels(labeled_foci, None, 10.0)
    
    del peaks
    del background_subtracted
    del spots
    del labeled_spots
    del labeled_whatever
    del expanded_labels
    del threshold_map
    del thresholded_image
    del image
    
    return output_labels

In [3]:
def extract_frame_features(Measurements : pd.DataFrame, image_name : str, pixel_volume : float):
    frame_features = pd.DataFrame({'Image_name' : [image_name]})
    frame_features = frame_features.assign(
        
        #Number of nuclei
        Foci_count = Measurements['label'].count(),

        
        #Nuclei Volume stats
        Size_mean  = Measurements['area'].mean() * pixel_volume,
        Size_std   = Measurements['area'].std() * pixel_volume,
        
    )
    return frame_features

## Folder processing

In [4]:
stack_folder_name = '#4695 - 06.06.2023'
dataset_folder = r'C:\Nisarg\Thesis\Data'
list_of_cell_summary_dataframes = []

In [5]:
stack_folder = os.path.join(dataset_folder, stack_folder_name)

measurements_folder = os.path.join(stack_folder, 'Measurements')
if not os.path.exists(measurements_folder):
    os.mkdir(measurements_folder)
    
Summary_filepath = os.path.join(stack_folder, stack_folder_name  + '_Summary.csv')
print(stack_folder, '\n', measurements_folder, '\n', Summary_filepath)

C:\Nisarg\Thesis\Data\#4695 - 06.06.2023 
 C:\Nisarg\Thesis\Data\#4695 - 06.06.2023\Measurements 
 C:\Nisarg\Thesis\Data\#4695 - 06.06.2023\#4695 - 06.06.2023_Summary.csv


In [6]:
file_names = os.listdir(stack_folder)

file_names_filtered = []
for file_name in file_names:
    if file_name.find('airy') >=0 and file_name.find('.czi') >= 0:
        file_names_filtered.append(file_name)
file_names_filtered

['#4695_11_W100_native_zstack_0.2%_airy.czi',
 '#4695_12_W100_native_zstack_0.2%_airy.czi',
 '#4695_13_W100_native_zstack_0.3%_airy.czi',
 '#4695_14_W100_native_zstack_0.2%_airy.czi']

## Batch Processing

In [7]:
#Threshold value for segmentation | 0.04 for WT and WT/E82X | 0.08 for E82X
threshold = 0.04

In [9]:
for file_name in file_names_filtered:
    
    #Reading image data    
    image_name = file_name
    aics_image = AICSImage(os.path.join(stack_folder, image_name))
    image = aics_image.get_image_data("ZYX", T=0, C=0)
    Voxel_size_z = aics_image.physical_pixel_sizes.Z
    Voxel_size_y = aics_image.physical_pixel_sizes.Y
    Voxel_size_x = aics_image.physical_pixel_sizes.X
    
    
    #Normalization
    image = image/image.max()
    
    #Rescale to isotropic
    zoom_factor = 3
    rescaled = cle.scale(image, factor_x= zoom_factor*(Voxel_size_x/Voxel_size_z), factor_y= zoom_factor*(Voxel_size_y/Voxel_size_z), factor_z= zoom_factor*(1.0), auto_size= True)
    pixel_volume = pow((Voxel_size_z / zoom_factor), 3)
    del image

    #Comute and save labels
    labels = segmentation(rescaled, threshold, zoom_factor)
    
    print(image_name)
    
    features = cle.statistics_of_labelled_pixels(label_image=labels)
    features = pd.DataFrame(features)
    features = features[['label', 'area' ]]
    del labels
    
    features_filename = image_name + '_measurements.csv'
    features_path = os.path.join(measurements_folder, features_filename)
    features.to_csv(features_path, sep=',', index_label='index')
    
    cell_summary = extract_frame_features(features, image_name, pixel_volume)
    list_of_cell_summary_dataframes.append(cell_summary)
    

#4695_11_W100_native_zstack_0.2%_airy.czi
#4695_12_W100_native_zstack_0.2%_airy.czi
#4695_13_W100_native_zstack_0.3%_airy.czi
#4695_14_W100_native_zstack_0.2%_airy.czi


In [10]:
list_of_cell_summary_dataframes

[                                  Image_name  Foci_count  Size_mean  Size_std
 0  #4695_11_W100_native_zstack_0.2%_airy.czi         737   0.038238   0.01527,
                                   Image_name  Foci_count  Size_mean  Size_std
 0  #4695_12_W100_native_zstack_0.2%_airy.czi         217   0.024786  0.011205,
                                   Image_name  Foci_count  Size_mean  Size_std
 0  #4695_13_W100_native_zstack_0.3%_airy.czi         389   0.021892  0.009796,
                                   Image_name  Foci_count  Size_mean  Size_std
 0  #4695_14_W100_native_zstack_0.2%_airy.czi         124   0.025615  0.009123]

# Run at the end of processing whole folder

In [11]:
folder_summary = pd.concat(list_of_cell_summary_dataframes, ignore_index=True)
folder_summary

,Image_name,Foci_count,Size_mean,Size_std
0,#4695_11_W100_native_zstack_0.2%_airy.czi,737,0.038238,0.015270
1,#4695_12_W100_native_zstack_0.2%_airy.czi,217,0.024786,0.011205
2,#4695_13_W100_native_zstack_0.3%_airy.czi,389,0.021892,0.009796
3,#4695_14_W100_native_zstack_0.2%_airy.czi,124,0.025615,0.009123


In [12]:
#Sorting list based on number, transfection and autophagy condition
folder_summary =pd.DataFrame(
    sorted(folder_summary.values, key=lambda x: x[0].split('_')[1:4][::-1]), 
    columns=folder_summary.columns
)
folder_summary

,Image_name,Foci_count,Size_mean,Size_std
0,#4695_11_W100_native_zstack_0.2%_airy.czi,737,0.038238,0.015270
1,#4695_12_W100_native_zstack_0.2%_airy.czi,217,0.024786,0.011205
2,#4695_13_W100_native_zstack_0.3%_airy.czi,389,0.021892,0.009796
3,#4695_14_W100_native_zstack_0.2%_airy.czi,124,0.025615,0.009123


In [12]:
folder_summary.to_csv(Summary_filepath, sep=',', index_label='index')